In [ ]:
!python -m pip install --user pyarrow

In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!curl "https://echanges.dila.gouv.fr/OPENDATA/ACCO/Freemium_acco_global_20220417-170000.tar.gz" -o "accords.tar.gz"

In [ ]:
!tar -xzvf "accords.tar.gz"

In [ ]:
!mkdir accords 

In [ ]:
!find acco -name '*.docx' -exec mv {} accords \;  

In [ ]:
import pandas as pd
import docx
import os

In [ ]:
DIR_DOCX = "accords\"
OUTPUT_FILE_PARQUET = "my_file.parquet"

In [ ]:
df = pd.DataFrame(columns=["num_dossier", "accorddocx"],dtype=object)

for i, num_dossier in enumerate(
    {file_name[:12] for file_name in os.listdir(DIR_DOCX) if file_name.endswith(".docx")}
):
    all_docx_files_for_num_dossier = sorted(
        [
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".docx") and file_name.startswith(num_dossier)
        ],
        key=os.path.getsize,
        reverse=True,
    )

    new_line = [num_dossier] + [
        "".join(open(file, "r").readlines()) for file in all_docx_files_for_num_dossier
    ]
    new_line = new_line[:2] if len(new_line) >= 2 else new_line + [""]
    try:
        new_line = {
            "num_dossier": new_line[0],
            "accorddocx": new_line[1],
            "index" : [i]
        }
    except:
        print(f"bug {num_dossier}")
        
    
    df_new_row = pd.DataFrame.from_records(data=new_line)
    df = pd.concat([df, df_new_row])

In [ ]:
df.to_parquet(OUTPUT_FILE_PARQUET)